In [35]:
# pip install emoji


   ---------------------------------------- 0.0/608.4 kB ? eta -:--:--
   ---------------------------------------- 608.4/608.4 kB 5.6 MB/s eta 0:00:00


In [37]:
import duckdb, os
import pandas as pd
import regex as re
import emoji

In [5]:
# Configuration and connection
DB_PATH = "D:/db/meta.duckdb"                 

con = duckdb.connect(DB_PATH)

con.execute("PRAGMA threads=2;")                         
con.execute("SET memory_limit='5GB';")                   
con.execute("SET preserve_insertion_order=false;")       

print("\n Set up ready")


 Set up ready


In [9]:
# Check a Parquet file columns 

con.sql("""
CREATE OR REPLACE VIEW sample_month AS
SELECT *
FROM "D:/dataset/meta_clean/year=2015/month=1/data_0.parquet"
""")

print(con.sql("PRAGMA table_info('sample_month');").fetchdf())

    cid              name       type  notnull dflt_value     pk
0     0          filename    VARCHAR    False       None  False
1     1          username    VARCHAR    False       None  False
2     2        like_count    INTEGER    False       None  False
3     3     comment_count    INTEGER    False       None  False
4     4             width    INTEGER    False       None  False
5     5            height    INTEGER    False       None  False
6     6          time_utc  TIMESTAMP    False       None  False
7     7           caption    VARCHAR    False       None  False
8     8      aspect_ratio     DOUBLE    False       None  False
9     9              area    INTEGER    False       None  False
10   10       orientation    VARCHAR    False       None  False
11   11          date_day       DATE    False       None  False
12   12               dow     BIGINT    False       None  False
13   13          hour_utc     BIGINT    False       None  False
14   14       has_caption    BOOLEAN    

In [39]:
# Regex pattern Unicode
HASHTAG_RE = re.compile(r'#[\p{L}\p{N}_]+')
MENTION_RE = re.compile(r'@[\p{L}\p{N}_]+')
URL_RE = re.compile(r'(?:http|https)://[^\s]+')

def count_emojis(text):
    if not isinstance(text, str):
        return 0
    return sum(char in emoji.EMOJI_DATA for char in text)

def enrich_and_save(year, month):
    # Percorsi input/output
    in_file = os.path.join(BASE_PATH, f"year={year}", f"month={month}", "data_0.parquet")
    out_dir = os.path.join(OUT_PATH, f"year={year}", f"month={month}")
    os.makedirs(out_dir, exist_ok=True)
    out_file = os.path.join(out_dir, "data_0.parquet")

    # Leggi parquet in pandas via DuckDB
    df = con.execute(f"SELECT * FROM '{in_file.replace('\\','/')}'").fetchdf()

    # Calcola feature in pandas
    def safe_count(pattern, text):
        if not isinstance(text, str) or not text:
            return 0
        return len(pattern.findall(text))

    df["n_hashtags"] = df["caption"].apply(lambda x: safe_count(HASHTAG_RE, x))
    df["n_mentions"] = df["caption"].apply(lambda x: safe_count(MENTION_RE, x))
    df["n_urls"] = df["caption"].apply(lambda x: safe_count(URL_RE, x))
    df["n_emojis"] = df["caption"].apply(count_emojis)

    # Salva come parquet
    df.to_parquet(out_file, index=False)
    print(f"Saved enriched file → {out_file}")

# Loop anni/mesi
for year in range(2012, 2020):
    if year == 2012:
        months = range(2, 13)
    elif year == 2019:
        months = range(1, 6)
    else:
        months = range(1, 13)

    for month in months:
        enrich_and_save(year, month)

Saved enriched file → D:/dataset/meta_before_join\year=2012\month=2\data_0.parquet
Saved enriched file → D:/dataset/meta_before_join\year=2012\month=3\data_0.parquet
Saved enriched file → D:/dataset/meta_before_join\year=2012\month=4\data_0.parquet
Saved enriched file → D:/dataset/meta_before_join\year=2012\month=5\data_0.parquet
Saved enriched file → D:/dataset/meta_before_join\year=2012\month=6\data_0.parquet
Saved enriched file → D:/dataset/meta_before_join\year=2012\month=7\data_0.parquet
Saved enriched file → D:/dataset/meta_before_join\year=2012\month=8\data_0.parquet
Saved enriched file → D:/dataset/meta_before_join\year=2012\month=9\data_0.parquet
Saved enriched file → D:/dataset/meta_before_join\year=2012\month=10\data_0.parquet
Saved enriched file → D:/dataset/meta_before_join\year=2012\month=11\data_0.parquet
Saved enriched file → D:/dataset/meta_before_join\year=2012\month=12\data_0.parquet
Saved enriched file → D:/dataset/meta_before_join\year=2013\month=1\data_0.parquet
S

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Saved enriched file → D:/dataset/meta_before_join\year=2018\month=1\data_0.parquet


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Saved enriched file → D:/dataset/meta_before_join\year=2018\month=2\data_0.parquet


...

Saved enriched file → D:/dataset/meta_before_join\year=2019\month=2\data_0.parquet


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Saved enriched file → D:/dataset/meta_before_join\year=2019\month=3\data_0.parquet


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Saved enriched file → D:/dataset/meta_before_join\year=2019\month=4\data_0.parquet
Saved enriched file → D:/dataset/meta_before_join\year=2019\month=5\data_0.parquet


In [40]:
# CHECKS

con.sql("""
CREATE OR REPLACE VIEW sample_month AS
SELECT *
FROM "D:/dataset/meta_before_join/year=2015/month=1/data_0.parquet";
""")

print(con.sql("""
SELECT username, caption, has_caption, caption_len_char, n_hashtags, n_urls, n_mentions, n_emojis
FROM sample_month
WHERE caption IS NOT NULL
LIMIT 10;
"""))

# Check hashtags
print(con.sql("""SELECT caption, n_hashtags
FROM sample_month
WHERE caption ILIKE '%#%' 
LIMIT 5;
"""))

# Check mentions
print(con.sql("""SELECT caption, n_mentions
FROM sample_month
WHERE caption ILIKE '%@%' 
LIMIT 5;
"""))

# Check urls
print(con.sql("""
SELECT caption, n_urls
FROM sample_month
WHERE caption ILIKE '%http%'
LIMIT 5;
"""))

# Check emojis
print(con.sql("""SELECT caption, n_emojis
FROM sample_month
WHERE n_emojis >= 2
LIMIT 5;
"""))

┌────────────┬───────────────────────────────────────────────────────────────────┬─────────────┬──────────────────┬────────────┬────────┬────────────┬──────────┐
│  username  │                              caption                              │ has_caption │ caption_len_char │ n_hashtags │ n_urls │ n_mentions │ n_emojis │
│  varchar   │                              varchar                              │   boolean   │      int64       │   int64    │ int64  │   int64    │  int64   │
├────────────┼───────────────────────────────────────────────────────────────────┼─────────────┼──────────────────┼────────────┼────────┼────────────┼──────────┤
│ 1nonlycash │ My loves❤️                                                         │ true        │               10 │          0 │      0 │          0 │        1 │
│ 1nonlycash │ Happy Founder's Day Sawrahs! #J13 #DST🐘🐤👠                      │ true        │               41 │          2 │      0 │          0 │        3 │
│ 1nonlycash │ Life only gets 

In [43]:
con.close()
print("Connection closed")

Connection closed
